In [3]:
import pandas as pd
import numpy as np
from pybedtools import BedTool

In [5]:
## Read in Illumina array CSV with locations, dropNA and set ints as str
illumina450kfile = "HumanMethylation450_15017482_v1-2.csv"
array450k = pd.read_csv(illumina450kfile, dtype={"CHR": str}, header = 7, usecols = (0,10,11,12))
array450k = array450k.dropna()
array450k[['MAPINFO', 'Genome_Build']] = array450k[['MAPINFO', 'Genome_Build']].astype(int)
## Extract locations with genome build GRCh37
array450k = array450k[array450k['Genome_Build'] == 37]
array450k = array450k.drop(['Genome_Build'], axis = 1)
## Drop X and Y
array450k = array450k.loc[~array450k['CHR'].isin(['X','Y'])]
array450k[['CHR', 'MAPINFO']] = array450k[['CHR', 'MAPINFO']].astype(int)
## Rearrange to represent a bed file
array450k["MAPINFO_Stop"] = array450k["MAPINFO"].astype(int) + 1  #BEDfiles are start 0-based and end 1-based
array450k = array450k[["CHR", "MAPINFO", "MAPINFO_Stop", "IlmnID"]]
array450k.sort_values(by = ["CHR", "MAPINFO"], inplace=True)
array450k.rename(index=str, columns={"CHR": "chr", "MAPINFO": "start", "MAPINFO_Stop": "stop", "IlmnID": "ilmnid"}, inplace=True)
## Print a row with 1s (= number of IlmnIDs in on that locus)
array450k["cpg_number"] = 1

#convert to a pybedtools object
array450k_bedobj = BedTool.from_dataframe(array450k)

print("The number of lines in the 450k file is: %i" % array450k_bedobj.count())
array450k_bedobj.head()

The number of lines in the 450k file is: 470870
1	15865	15866	cg13869341	1
 1	18827	18828	cg14008030	1
 1	29407	29408	cg12045430	1
 1	29425	29426	cg20826792	1
 1	29435	29436	cg00381604	1
 1	68849	68850	cg20253340	1
 1	69591	69592	cg21870274	1
 1	91550	91551	cg03130891	1
 1	135252	135253	cg24335620	1
 1	449076	449077	cg16162899	1
 

In [6]:
## Load in the theoretical covered RRBS regions (made with mkrr2genome)
RRBSfile = "RRBS_regions20-200.bed"
RRBS = pd.read_csv(RRBSfile, sep = "\t", header = None)
RRBS.rename(index=str, columns={0: "chr", 1: "start", 2: "stop"}, inplace=True)

print("The number of lines in the RRBS file is: %i" % RRBS.shape[0])
RRBS.head()

/Users/rmvpaeme/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


The number of lines in the RRBS file is: 725894


,chr,start,stop
0,1,10497,10588
1,1,10589,10640
2,1,10641,10669
3,1,10670,10698
4,1,10699,10727


In [8]:
## In the RRBS file, the stop position is sometimes only 1 nt before the next start position.
## Merge these to reduce the number of regions
RRBS_bedobj = BedTool.from_dataframe(RRBS)
RRBS_merged = RRBS_bedobj.merge(d=1)
print("The number of lines in the RRBS file is: %i" % RRBS_merged.count())
RRBS_merged.head()

The number of lines in the RRBS file is: 403572
1	10497	11479
 1	11763	11959
 1	12271	12354
 1	12586	12778
 1	14791	14887
 1	15089	15188
 1	15720	16081
 1	16822	16973
 1	17562	17852
 1	21002	21023
 

In [9]:
## Find the intersecting regions between the RRBS and 450k file, including the number of CpGs that are in the intersecting region
RRBS_and_450k = RRBS_merged.intersect(array450k_bedobj, wa=True, c = True)
print("The number of lines in the merged RRBS and 450k file is: %i" % RRBS_and_450k.count())
RRBS_and_450k.head()


The number of lines in the merged RRBS and 450k file is: 403572
1	10497	11479	0
 1	11763	11959	0
 1	12271	12354	0
 1	12586	12778	0
 1	14791	14887	0
 1	15089	15188	0
 1	15720	16081	1
 1	16822	16973	0
 1	17562	17852	0
 1	21002	21023	0
 

In [10]:
## Back to pandas dataframe
RRBS_450k_intersect = pd.read_table(RRBS_and_450k.fn, names=['chrom', 'start', 'stop', 'cpg_in_450k'])

## Remove all RRBS regions that are spanning less than 3 CpGs on the microarray
RRBS_450k_intersect = RRBS_450k_intersect.loc[~RRBS_450k_intersect['cpg_in_450k'].isin(['0','1','2'])]

print("The number of lines in the merged RRBS and 450k file after removing 0-2 CpG in 450k file: %i" % RRBS_450k_intersect.shape[0])
print("The number of CpGs from the 450k array covered by this method: %i" % RRBS_450k_intersect['cpg_in_450k'].sum())
RRBS_450k_intersect.head()

The number of lines in the merged RRBS and 450k file after removing 0-2 CpG in 450k file: 14103
The number of CpGs from the 450k array covered by this method: 61750


/Users/rmvpaeme/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,chrom,start,stop,cpg_in_450k
14,1,29165,29483,3
61,1,567122,567401,4
88,1,762982,763181,3
112,1,844314,845703,3
131,1,859816,860118,3


In [11]:
## Drop the number column
RRBS_450k_intersect.drop("cpg_in_450k", axis = 1, inplace = True)

In [12]:
## Add a new column with the clusterID number
RRBS_450k_intersect = RRBS_450k_intersect.reset_index(drop=True)
RRBS_450k_intersect['clusterID'] = range(1, len(RRBS_450k_intersect)+1)

In [13]:
RRBS_450k_intersect.head()

,chrom,start,stop,clusterID
0,1,29165,29483,1
1,1,567122,567401,2
2,1,762982,763181,3
3,1,844314,845703,4
4,1,859816,860118,5


In [150]:
RRBS_450k_intersect.to_csv("RRBS_450k_intersectClusters.tsv", index = None, sep = "\t")